In [1]:
from time import sleep
import requests
import numpy as np
import pandas as pd
from panoramix.decompiler import decompile_bytecode
import multiprocessing
import re


In [ ]:
def soft_decompile(byte_code):
    try:
        decompiled_code = decompile_bytecode(byte_code).text
        assert len(decompiled_code) > 200
        return decompiled_code
    except:
        try:
            byte_code = '0x' + byte_code[66:]
            decompiled_code = decompile_bytecode(byte_code).text
            assert len(decompiled_code) > 200
            return decompiled_code
        except:
            return "error"


def force_web3_code_decompile(contract_address):
    print("load from web3...")
    web3 = Web3(Web3.HTTPProvider("https://eth-mainnet.gateway.pokt.network/v1/lb/6266d6cdaa777e00391e0d29"))
    contract_bytecode = web3.eth.getCode(Web3.toChecksumAddress(contract_address))
    contract_bytecode_string = contract_bytecode.hex()
    decompiled_code = decompile_bytecode(contract_bytecode_string).text
    return decompiled_code

def simple_web3_code_decompile(byte_code):
    try:
        return decompile_bytecode(byte_code).text
    except:
        return "error"


def preprocess_code(code_str):
    code_str = code_str.replace("\n\n", "\n")
    lines = code_str.split('\n')

    # for line in lines:
    #     print(line)

    code_block = False
    for i, line in enumerate(lines):

        lines[i] += " ;"
        line += " ;"
        if line[:2] == "  " and not code_block:
            code_block = True
            lines[i] = line[2:]
            lines[i - 1] = lines[i - 1].replace(':  ;', ' {')
            lines[i - 1] = lines[i - 1].replace(': ;', ' {')
        elif line[:2] == "  " and code_block:
            lines[i] = line[2:]
        elif line[:2] != "  " and code_block:
            code_block = False
            lines[i - 1] = lines[i - 1] + ' }'

        lines[i] = lines[i].replace('(', ' ( ')
        lines[i] = lines[i].replace(')', ' )')
        lines[i] = lines[i].replace('[', ' [ ')
        lines[i] = lines[i].replace(']', ' ]')
        lines[i] = lines[i].replace('.', ' . ')
        lines[i] = lines[i].replace('=', ' = ')

    code_block = False

    blocks_truncated = False
    while not blocks_truncated:
        blocks_truncated = True
        for i, line in enumerate(lines):
            if line[:2] == "  " and not code_block:
                code_block = True
                blocks_truncated = False
                lines[i] = line[2:]
                lines[i - 1] = lines[i - 1].replace(':  ;', ' {')
                lines[i - 1] = lines[i - 1].replace(': ;', ' {')
            elif line[:2] == "  " and code_block:
                lines[i] = line[2:]
            elif line[:2] != "  " and code_block:
                code_block = False
                lines[i - 1] = lines[i - 1] + ' }'

    blocks_truncated = False
    while not blocks_truncated:
        blocks_truncated = True
        for i, line in enumerate(lines):
            if line[:2] == " " and not code_block:
                code_block = True
                blocks_truncated = False
                lines[i] = line[2:]
                lines[i - 1] = lines[i - 1].replace(':  ;', ' {')
                lines[i - 1] = lines[i - 1].replace(': ;', ' {')
            elif line[:2] == " " and code_block:
                lines[i] = line[2:]
            elif line[:2] != " " and code_block:
                code_block = False
                lines[i - 1] = lines[i - 1] + ' }'

    for i, line in enumerate(lines):
        lines[i] = line.replace("  ", " ")
    # for line in lines:
    #     print(line)

    joined =  " ".join(lines)
    pattern_code = r"0x[A-Fa-f0-9]{41,}\s"
    pattern_address = r"0x[A-Fa-f0-9]{40}\s"
    result = re.sub(pattern_code, "CODE", joined)
    result = re.sub(pattern_address, "ADDRESS", result)

    return result

global count
count = 0
def timeout_handler(signum, frame):
    raise TimeoutError("Timed out!")

def decompile_and_preprocess_row_target(bytecode_from_web3, creation_bytecode, return_dict):
    try:

        # if count % 10 == 0:
        #     print(count)

        if bytecode_from_web3 == '0x':
            return_dict['result'] = preprocess_code(soft_decompile(creation_bytecode))
        else:
            return_dict['result'] = preprocess_code(simple_web3_code_decompile(bytecode_from_web3))
    except:
        return_dict['result'] = "error"


def decompile_and_preprocess_row(bytecode_from_web3, creation_bytecode, malicious):
    if malicious == 0:
        manager = multiprocessing.Manager()
        return_dict = manager.dict()

        p = multiprocessing.Process(target=decompile_and_preprocess_row_target, args=(bytecode_from_web3, creation_bytecode, return_dict))
        p.start()
        p.join(60)
        if p.is_alive():
            print("running... let's kill it...")
            p.terminate()
            p.join()
            return "error"
        else:
            return return_dict['result']
    elif malicious == 1:
        return_dict = {}
        decompile_and_preprocess_row_target(bytecode_from_web3, creation_bytecode, return_dict)
        return return_dict['result']
    else:
        return "error"


In [2]:


LUABASE_API_KEY = "sk-live-j3yb8p1404-2e4631c8-b914-4b23-aada-0b2a2441db64"
LUABASE_QUERY_URL = "https://q.luabase.com/run"
MALICIOUS_CONTRACT_SQL = '''
SELECT *
FROM forta.malicious_contract_detection_dataset
ORDER BY malicious DESC
'''

def get_luabase_data(sql: str, limit: int = 30000) -> pd.DataFrame:
    payload = {
        "block": {
            "details": {
                "sql": sql,
                "limit": limit,
                "parameters": {}
            }
        },
        "api_key": LUABASE_API_KEY,
    }
    headers = {"content-type": "application/json"}
    response = requests.request("POST", LUABASE_QUERY_URL, json=payload, headers=headers)
    data = response.json()
    return pd.DataFrame(data['data'])


sc_df = get_luabase_data(MALICIOUS_CONTRACT_SQL)

In [3]:
sc_df.to_csv('/home/venglov/Documents/scdf.csv')

In [2]:
sc_df = pd.read_csv('/home/venglov/Documents/scdf.csv')

In [3]:
from web3 import Web3
web3 = Web3(Web3.HTTPProvider("https://eth-mainnet.gateway.pokt.network/v1/lb/6266d6cdaa777e00391e0d29"))
global counter
counter = 0

def request_byte_code_from_web3(contract_address):
    global counter
    try:
        counter = counter+1
        if counter % 10 == 0:
            print(counter)
        contract_bytecode = web3.eth.getCode(Web3.toChecksumAddress(contract_address))
        contract_bytecode_string = contract_bytecode.hex()

        return contract_bytecode_string
    except:
        try:
            print("sleep 2s")
            sleep(2)
            contract_bytecode = web3.eth.getCode(Web3.toChecksumAddress(contract_address))
            contract_bytecode_string = contract_bytecode.hex()

            return contract_bytecode_string
        except:
            return ""

In [4]:
sc_df['bytecode_from_web3'] = sc_df.apply(lambda row: request_byte_code_from_web3(row.contract_address), axis = 1)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
sleep 2s
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200

In [6]:
sc_df.to_csv('/home/venglov/Documents/scdf_with_web3bc.csv')

In [4]:
sc_df = pd.read_csv('/home/venglov/Documents/scdf_with_web3bc.csv')

In [8]:
malicious_df = sc_df.drop(sc_df[sc_df.malicious == 0].index)
malicious_df.to_csv('/home/venglov/Documents/malicious.csv')

In [9]:
normal_df = sc_df.drop(sc_df[sc_df.malicious == 1].index)
normal_df.to_csv('/home/venglov/Documents/normal_df.csv')

In [ ]:
# sc_df = pd.read_csv('/home/venglov/Documents/scdf_with_web3bc.csv')
# sc_df = sc_df[:1000]
# sc_df['decompiled_code'] = ""
# for i in range(len(sc_df)):
#     # if isinstance(sc_df.decompiled_code[i], str):
#     #     continue
#     print(f'Number: {i}, address: {sc_df.contract_address[i]}')
#     sc_df.decompiled_code[i] = decompile_and_preprocess_row(sc_df.bytecode_from_web3[i], sc_df.creation_bytecode[i], sc_df.malicious[i])
#     if i % 10 == 0:
#         sc_df.to_csv('/home/venglov/Documents/scdf_with_code_lite4.csv')
#
#
#
# sc_df['decompiled_code'] = sc_df.apply(lambda row: decompile_and_preprocess_row(row.bytecode_from_web3, row.creation_bytecode), axis = 1)

In [ ]:
# sc_df = pd.read_csv('/home/venglov/Documents/scdf_with_code_lite2.csv')
# for i in range(len(sc_df)):
#     if isinstance(sc_df.decompiled_code[i], str) and sc_df.decompiled_code[i] != "# No code found for this contract . ;":
#         continue
#     print(f'Number: {i}, address: {sc_df.contract_address[i]}')
#     sc_df.decompiled_code[i] = decompile_and_preprocess_row(sc_df.bytecode_from_web3[i], sc_df.creation_bytecode[i], sc_df.malicious[i])
#     if i % 10 == 0:
#         sc_df.to_csv('/home/venglov/Documents/scdf_with_code_lite.csv')


In [ ]:
# sc_df = pd.read_csv('/home/venglov/Documents/scdf_with_web3bc.csv')
# sc_df = sc_df[:1000]
# sc_df['decompiled_code'] = sc_df.apply(lambda row: decompile_and_preprocess_row(row.bytecode_from_web3, row.creation_bytecode), axis = 1)
# sc_df.to_csv('/home/venglov/Documents/scdf_with_code_lite_plus.csv')

In [ ]:
# sc_df['bytecode_len'] = sc_df.apply(lambda row: len(str(row.bytecode_from_web3)), axis = 1)
# rslt_df = sc_df[sc_df['malicious'] > 0]
# deleted_df = rslt_df[rslt_df['bytecode_from_web3'] == '0x']
# rslt_df['bytecode_len'] = rslt_df.apply(lambda row: len(str(row.bytecode_from_web3)), axis = 1)
# rslt_df = rslt_df.sort_values('bytecode_len', ascending=False)
# long_exploit = rslt_df.sort_values('bytecode_len', ascending=False).bytecode_from_web3[62]
# import swifter

# deleted_df['target_code'] = deleted_df.apply(lambda row: soft_decompile(row.creation_bytecode), axis = 1)


In [ ]:
malicious_df = pd.read_csv('/home/venglov/Documents/malicious.csv')
malicious_df['decompiled_code'] = ""
for i in range(len(malicious_df)):
    # if isinstance(sc_df.decompiled_code[i], str):
    #     continue
    print(f'Number: {i}, address: {malicious_df.contract_address[i]}')
    malicious_df.decompiled_code[i] = decompile_and_preprocess_row(malicious_df.bytecode_from_web3[i], malicious_df.creation_bytecode[i], malicious_df.malicious[i])
    if i % 10 == 0:
        malicious_df.to_csv('/home/venglov/Documents/malicious_with_code.csv')


In [ ]:
normal_df = pd.read_csv('/home/venglov/Documents/normal_df.csv')
normal_df = normal_df.sample(frac=1).reset_index(drop=True)
normal_df = normal_df[:3000]
normal_df['decompiled_code'] = ""
for i in range(len(normal_df)):
    # if isinstance(sc_df.decompiled_code[i], str):
    #     continue
    print(f'Number: {i}, address: {normal_df.contract_address[i]}')
    normal_df.decompiled_code[i] = decompile_and_preprocess_row(normal_df.bytecode_from_web3[i], normal_df.creation_bytecode[i], normal_df.malicious[i])
    if i % 10 == 0:
        normal_df.to_csv('/home/venglov/Documents/normal_with_code.csv')

normal_df.to_csv('/home/venglov/Documents/normal_with_code.csv')


In [16]:
%%time

def get_contract_as_image(x):
    scale = 16 ## equals to hexadecimal
    num_of_bits = 8
    x_bit = bin(int(x, scale))[2:].zfill(num_of_bits)
    np_image_code = np.fromstring(x_bit,'u1') - ord('0')
    if len(np_image_code) > 512 * 512:
        np_image_code = np_image_code[0:512*512]
        return np_image_code.reshape((512, 512, 1))
    additional_zeros = 512 * 512 - len(np_image_code)
    additional_zeros = additional_zeros if additional_zeros >= 0 else 0
    np_full_image = np.pad(np_image_code, (0, additional_zeros), 'constant')
    return np_full_image.reshape((512, 512, 1))

img = get_contract_as_image(sc_df.creation_bytecode[np.random.randint(0, len(sc_df))])


CPU times: user 0 ns, sys: 583 µs, total: 583 µs
Wall time: 780 µs


<timed exec>:5: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
